In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## Distance to board: 8.6 [m]

In [ ]:
path = '../data/fee_corridor/bags/leica_ouster/'

csvs = ['big_board/8m/depth_correction_black_board_2023-02-06-16-46-40_bias_estimation.csv',
        'big_board/8m/depth_correction_2023-02-06-16-15-50_bias_estimation.csv',
        'small_board/depth_correction_black_board_ouster_leica_2023-02-02-14-50-38_bias_estimation.csv',
        'small_board/depth_correction_white_board_ouster_leica_2023-02-02-13-44-08_bias_estimation.csv']

csvs = [os.path.join(path, f) for f in csvs]

In [ ]:
df = pd.read_csv(csvs[0])

In [ ]:
print([k for k, _ in df.items()])

In [ ]:
for f in csvs:
    df = pd.read_csv(f)
    
    plt.figure()
    plt.plot(df['# Angles [deg]'])
    plt.show()

In [ ]:
def interpolate_icp_dists(angles, dists, min_angle=0, max_angle=80, angle_step=1):
    
    angles_uniform = np.arange(min_angle, max_angle, angle_step)
    if angles[0] > angles[-1]:
        angles = angles[::-1]
        dists = dists[::-1]
        
    dists_interp = np.interp(angles_uniform, angles, dists)
    
    if np.min(angles) > min_angle:
        dists_interp[angles_uniform < np.min(angles)] = np.nan
        
    if np.max(angles) < max_angle:
        dists_interp[angles_uniform > np.max(angles)] = np.nan

    return angles_uniform, dists_interp

In [ ]:
from scipy.signal import argrelextrema

plt.figure(figsize=(10, 10))
plt.grid(visible=True)

all_angles = []
all_icp_dists = []

for f in csvs:
    df = pd.read_csv(f)
    
    angles = np.asarray(df['# Angles [deg]'])
    
    icp_dists = df[' ICP dist [m]']

    minimum_ids = argrelextrema(angles, np.less)[0]
    maximum_ids = argrelextrema(angles, np.greater)[0]

    extremum_ids = np.sort(minimum_ids.tolist() + maximum_ids.tolist()).tolist()   
    ids = [0] + extremum_ids + [len(angles)]

    for i in range(len(ids)-1):
        angles_seq = angles[ids[i]:ids[i+1]]
        dists_seq = icp_dists[ids[i]:ids[i+1]]
        
        angles_interp, dists_interp = interpolate_icp_dists(angles_seq, dists_seq)
        assert len(angles_interp) == len(dists_interp)
        
        plt.plot(angles_interp, dists_interp, '--')
        plt.plot(angles_seq, dists_seq, 'x')
        
        all_angles.append(angles_interp)
        all_icp_dists.append(dists_interp)
        
mean_dists = np.nanmean(all_icp_dists, axis=0)
std_dists = np.nanstd(all_icp_dists, axis=0)

plt.plot(angles_interp, mean_dists)
plt.fill_between(angles_interp, mean_dists - std_dists, mean_dists + std_dists, alpha=0.2)

In [ ]:
def mean_bias_estimation(csvs):
    all_angles = []
    all_icp_dists = []
    all_icp_dists_cor = []

    for f in csvs:
        df = pd.read_csv(f)

        angles = np.asarray(df['# Angles [deg]'])

        icp_dists = df[' ICP dist [m]']
        icp_dists_cor = df[' ICP dist DC [m]']

        minimum_ids = argrelextrema(angles, np.less)[0]
        maximum_ids = argrelextrema(angles, np.greater)[0]

        extremum_ids = np.sort(minimum_ids.tolist() + maximum_ids.tolist()).tolist()   
        ids = [0] + extremum_ids + [len(angles)]

        for i in range(len(ids)-1):
            angles_seq = angles[ids[i]:ids[i+1]]
            dists_seq = icp_dists[ids[i]:ids[i+1]]
            dists_cor_seq = icp_dists_cor[ids[i]:ids[i+1]]

            angles_interp, dists_interp = interpolate_icp_dists(angles_seq, dists_seq)
            _, dists_cor_interp = interpolate_icp_dists(angles_seq, dists_cor_seq)
            assert len(angles_interp) == len(dists_interp)

            all_angles.append(angles_interp)
            all_icp_dists.append(dists_interp)
            all_icp_dists_cor.append(dists_cor_interp)

    mean_dists = np.nanmean(all_icp_dists, axis=0)
    std_dists = np.nanstd(all_icp_dists, axis=0)
    
    mean_dists_cor = np.nanmean(all_icp_dists_cor, axis=0)
    std_dists_cor = np.nanstd(all_icp_dists_cor, axis=0)

    return mean_dists, std_dists, mean_dists_cor, std_dists_cor, angles_interp

In [ ]:
plt.rcParams.update({'font.size': 24})
plt.figure(figsize=(20, 10))
plt.grid()
# plt.ylim([0, 0.20])
# plt.xlim([0, 90])
plt.ylabel('ICP point-to-plane distance [m]')
plt.xlabel('Incidence angle [deg]')

def fname2label(fname):
    label_pattern = '{distance}m_dist_{board_side}_board'
    distance = 5.3 if 'small' in fname else 8.6
    board_side = 'black' if 'black' in fname else 'white'
    label = label_pattern.format(distance=distance, board_side=board_side)
    return label

np.random.seed(14)
colors = [np.random.random(3) for f in csvs]

for i, f in enumerate(csvs):
    mean_dists, std_dists, mean_dists_cor, std_dists_cor, angles_interp = mean_bias_estimation(csvs=[f])

    plt.plot(angles_interp, mean_dists, '--', color=colors[i])
    plt.fill_between(angles_interp, mean_dists - std_dists, mean_dists + std_dists, alpha=0.2, color=colors[i])
    
    plt.plot(angles_interp, mean_dists_cor, label=fname2label(f), color=colors[i])
    plt.fill_between(angles_interp, mean_dists_cor - std_dists_cor, mean_dists_cor + std_dists_cor, alpha=0.2, color=colors[i])
    plt.legend()